<a href="https://colab.research.google.com/github/RockYuan/DeepLearning/blob/main/ieee_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 映射GDrive目录

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 配置kaggle和下载数据集

In [ ]:
# from google.colab import files
# files.upload()

In [2]:
! mkdir ~/.kaggle
! cp /content/gdrive/MyDrive/.kaggle/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! ls -lh ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
-rw------- 1 root root 64 Apr  3 11:33 /root/.kaggle/kaggle.json


In [5]:
%cd /content/gdrive/MyDrive/ai-lake/
! kaggle competitions download -c 'ieee-fraud-detection'
! mkdir ieee-fraud-detection
! unzip ieee-fraud-detection.zip -d ieee-fraud-detection
# ! ls -lh ieee-fraud-detection


/content/gdrive/MyDrive/ai-lake
ieee-fraud-detection.zip: Skipping, found more recently modified local copy (use --force to force download)
mkdir: cannot create directory ‘ieee-fraud-detection’: File exists
Archive:  ieee-fraud-detection.zip
replace ieee-fraud-detection/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace ieee-fraud-detection/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ieee-fraud-detection/sample_submission.csv  
  inflating: ieee-fraud-detection/test_identity.csv  
  inflating: ieee-fraud-detection/test_transaction.csv  
  inflating: ieee-fraud-detection/train_identity.csv  
  inflating: ieee-fraud-detection/train_transaction.csv  
total 1.3G
-rw------- 1 root root 5.8M Dec 11  2019 sample_submission.csv
-rw------- 1 root root  25M Dec 11  2019 test_identity.csv
-rw------- 1 root root 585M Dec 11  2019 test_transaction.csv
-rw------- 1 root root  26M Dec 11  2019 train_identity.csv
-rw-----

# 安装autogluon在指定项目目录下

In [3]:
%cd /content/gdrive/MyDrive/ai-lake/ieee-fraud-detection

/content/gdrive/MyDrive/ai-lake/ieee-fraud-detection


In [8]:
!pip list | grep autogluon

In [ ]:
!pip install autogluon -t .

# 特征工程

In [1]:
%%time
%cd /content/gdrive/MyDrive/ai-lake/ieee-fraud-detection

import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

pd.options.display.max_columns = None
pd.set_option("display.max_rows", 50)

directory = '/content/gdrive/MyDrive/ai-lake/ieee-fraud-detection/'  # directory where you have downloaded the data CSV files from the competition
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
save_path = directory + 'ieee-fraud-detection-Models/'  # where to store trained models

/content/gdrive/MyDrive/ai-lake/ieee-fraud-detection
CPU times: user 1.21 s, sys: 454 ms, total: 1.66 s
Wall time: 5.16 s


In [3]:
%%time
train_identity = pd.read_csv(directory+'train_identity.csv')
train_transaction = pd.read_csv(directory+'train_transaction.csv')
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

train_data.shape

CPU times: user 28.2 s, sys: 6.16 s, total: 34.4 s
Wall time: 36.2 s


In [4]:
%%time
columnsList = train_data.columns.to_list()

for col in columnsList:
  train_data[col].fillna(-999, inplace=True)

CPU times: user 1.73 s, sys: 18.8 ms, total: 1.75 s
Wall time: 1.75 s


In [7]:
train_data[train_data['isFraud'] == 1].head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
203,2987203,1,89760,445.000000,W,18268,583.0,150.0,visa,226.0,credit,251.0,87.0,-999.0,-999.0,aol.com,-999,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,4.0,2.0,57.0,57.0,52.0,0.0,-999.0,-999.0,-999.0,-999.000000,-999.000000,80.0,-999.0,-999.0,-999.0,-999.0,374.0,-999,-999,-999,M0,F,T,-999,-999,-999,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.000000,-999.000000,-999.000000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.000000,-999.000000,-999.000000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.000000,-999.000000,-999.000000,-999.0,-999.0,-999.0,-999.0,-999.

In [ ]:
train_data.dtypes.to_dict()

In [6]:
%%time
for col in train_data.columns:
  if train_data[col].dtype=='float64':
    train_data[col] = train_data[col].astype('float32')
  if train_data[col].dtype=='int64':
    train_data[col] = train_data[col].astype('int32')

CPU times: user 1min 40s, sys: 3.39 s, total: 1min 43s
Wall time: 1min 55s


In [8]:
%%time
# train_data_fe = train_data.copy()
train_data.to_csv( directory + 'train_data_fe.gz', compression='gzip')

CPU times: user 5min 19s, sys: 1.25 s, total: 5min 20s
Wall time: 5min 24s


In [2]:
%%time
train_data_fe = pd.read_csv(directory+'train_data_fe.gz')
# train_data_fe.shape

CPU times: user 30.7 s, sys: 6.28 s, total: 37 s
Wall time: 37.6 s


In [ ]:
%%time
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path).fit(
    train_data_fe, time_limit=600
)

Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "/content/gdrive/MyDrive/ai-lake/ieee-fraud-detection/ieee-fraud-detection-Models/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    590540
Train Data Columns: 434
Label Column: isFraud
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    8583.34 MB
	Train Data (Original)  Memory Usage: 3029.09 MB (35.3% of available memory)
	Inferring data type of each feature based o

In [3]:
!nvidia-smi

Sun Apr  3 11:51:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------